In [2]:
import os
import tensorflow as tf
from PIL import Image
import numpy as np
from nets  import nets_factory

In [3]:
#字符数量
CHAR_NUM=10
#图片高度
IMAGE_HEIGHT=60
IMAGE_WIDTH=160
BATCH_SIZE=20
TFRECORD_FILE="D:/Tensorflow/captcha/train.tfrecords"

x=tf.placeholder(tf.float32,[None,224,224])
y0=tf.placeholder(tf.float32,[None])
y1=tf.placeholder(tf.float32,[None])
y2=tf.placeholder(tf.float32,[None])
y3=tf.placeholder(tf.float32,[None])

lr=tf.Variable(0.003,dtype=tf.float32)
print('1')
def  read_and_decode(filename):
    #根据文件名生成队列
    filename_queue=tf.train.string_input_producer([filename])
    reader=tf.TFRecordReader()
    #返回文件名和文件
    _, serialized_example=reader.read(filename_queue)
    features=tf.parse_single_example(serialized_example,features={'image':tf.FixedLenFeature([],tf.string),
                                                                  'label0':tf.FixedLenFeature([],tf.int64),
                                                                  'label1':tf.FixedLenFeature([],tf.int64),
                                                                  'label2':tf.FixedLenFeature([],tf.int64),
                                                                  'label3':tf.FixedLenFeature([],tf.int64)
                                                                 })
    image=tf.decode_raw(features['image'],tf.uint8)
    image=tf.reshape(image,[224,224])
    image=tf.cast(image,tf.float32)/255.0
    image=tf.subtract(image,0.5)
    image=tf.multiply(image,2.0)
    
    label0=tf.cast(features['label0'],tf.int32)
    label1=tf.cast(features['label1'],tf.int32)
    label2=tf.cast(features['label2'],tf.int32)
    label3=tf.cast(features['label3'],tf.int32)
    return image,label0,label1,label2,label3


1


In [4]:
image,label0,label1,label2,label3=read_and_decode(TFRECORD_FILE)
image_batch,label_batch0,label_batch1,label_batch2,label_batch3=tf.train.shuffle_batch(
    [image,label0,label1,label2,label3],batch_size=BATCH_SIZE,capacity=1075,min_after_dequeue=1000,num_threads=1)
train_network_fn=nets_factory.get_network_fn(
  'alexnet_v2',
  num_classes=CHAR_NUM,
  weight_decay=0.0005,
  is_training=True)
gpu_options = tf.GPUOptions(allow_growth=True)
with tf.Session(config=tf.ConfigProto(log_device_placement=False,allow_soft_placement=True,gpu_options=gpu_options)) as sess:
#     gpu_options = tf.GPUOptions(allow_growth=True)
#     tf.Session(config=tf.ConfigProto(log_device_placement=False,allow_soft_placement=True,gpu_options=gpu_options))
    X=tf.reshape(x,[BATCH_SIZE,224,224,1])
    logits0,logits1,logits2,logits3,end_pintos=train_network_fn(X)
    one_hot_labels0=tf.one_hot(indices=tf.cast(y0,tf.int32),depth=CHAR_NUM)
    one_hot_labels1=tf.one_hot(indices=tf.cast(y1,tf.int32),depth=CHAR_NUM)
    one_hot_labels2=tf.one_hot(indices=tf.cast(y2,tf.int32),depth=CHAR_NUM)
    one_hot_labels3=tf.one_hot(indices=tf.cast(y3,tf.int32),depth=CHAR_NUM)
    
    loss0=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits0,labels=one_hot_labels0))
    loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits1,labels=one_hot_labels1))
    loss2=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits2,labels=one_hot_labels2))
    loss3=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits3,labels=one_hot_labels3))
    total_loss=(loss0+loss1+loss2+loss3)/4.0
    optimizer=tf.train.AdamOptimizer(learning_rate=lr).minimize(total_loss)
    
    correct_prediction0=tf.equal(tf.argmax(one_hot_labels0,1),tf.argmax(logits0,1))
    accuracy0=tf.reduce_mean(tf.cast(correct_prediction0,tf.float32))
    
    correct_prediction1=tf.equal(tf.argmax(one_hot_labels1,1),tf.argmax(logits1,1))
    accuracy1=tf.reduce_mean(tf.cast(correct_prediction1,tf.float32))
    
    correct_prediction2=tf.equal(tf.argmax(one_hot_labels2,1),tf.argmax(logits2,1))
    accuracy2=tf.reduce_mean(tf.cast(correct_prediction2,tf.float32))
    
    correct_prediction3=tf.equal(tf.argmax(one_hot_labels3,1),tf.argmax(logits3,1))
    accuracy3=tf.reduce_mean(tf.cast(correct_prediction3,tf.float32))
    
    saver=tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    coord=tf.train.Coordinator()  
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    for i in range(10001):
        b_image,b_label0,b_label1,b_label2,b_label3=sess.run([image_batch,label_batch0,label_batch1,label_batch2,label_batch3])
        sess.run(optimizer,feed_dict={x:b_image,y0:b_label0,y1:b_label1,y2:b_label2,y3:b_label3})       
        if i%20==0:
            if i%2000==0:
                sess.run(tf.assign(lr,lr/3))
            acc0,acc1,acc2,acc3,loss_=sess.run([accuracy0,accuracy1,accuracy2,accuracy3,total_loss],feed_dict={x:b_image,                                                                                                y0:b_label0,
                                                                                                          y1:b_label1,
                                                                                                          y2:b_label2,
                                                                                                          y3:b_label3})
            learning_rate=sess.run(lr)
            print("Iter:%d Loss:%.3f Accuracy:%.2f,%.2f,%.2f,%.2f  Learning_rate:%.4f"%(i,loss_,acc0,acc1,acc2,acc3,learning_rate))
            if i==10000:  
                saver.save(sess,'./captcha/models/crack_captcha.model',global_step=1)
                break
    coord.request_stop()
    coord.join(threads)

Iter:0 Loss:2171.862 Accuracy:0.25,0.25,0.20,0.30  Learning_rate:0.0010
Iter:20 Loss:2.307 Accuracy:0.15,0.15,0.00,0.05  Learning_rate:0.0010
Iter:40 Loss:2.285 Accuracy:0.10,0.25,0.25,0.20  Learning_rate:0.0010
Iter:60 Loss:2.301 Accuracy:0.20,0.05,0.00,0.25  Learning_rate:0.0010
Iter:80 Loss:2.304 Accuracy:0.10,0.05,0.05,0.00  Learning_rate:0.0010
Iter:100 Loss:2.305 Accuracy:0.15,0.05,0.10,0.10  Learning_rate:0.0010
Iter:120 Loss:2.304 Accuracy:0.15,0.15,0.15,0.05  Learning_rate:0.0010
Iter:140 Loss:2.296 Accuracy:0.10,0.10,0.15,0.10  Learning_rate:0.0010
Iter:160 Loss:2.309 Accuracy:0.20,0.15,0.10,0.15  Learning_rate:0.0010
Iter:180 Loss:2.299 Accuracy:0.10,0.05,0.10,0.10  Learning_rate:0.0010
Iter:200 Loss:2.302 Accuracy:0.10,0.15,0.25,0.00  Learning_rate:0.0010
Iter:220 Loss:2.310 Accuracy:0.10,0.05,0.15,0.05  Learning_rate:0.0010
Iter:240 Loss:2.295 Accuracy:0.00,0.05,0.15,0.05  Learning_rate:0.0010
Iter:260 Loss:2.306 Accuracy:0.10,0.10,0.15,0.15  Learning_rate:0.0010
Iter:280 

Iter:2300 Loss:0.675 Accuracy:0.95,0.75,0.70,0.70  Learning_rate:0.0003
Iter:2320 Loss:0.697 Accuracy:1.00,0.85,0.45,0.80  Learning_rate:0.0003
Iter:2340 Loss:0.818 Accuracy:0.90,0.70,0.60,0.75  Learning_rate:0.0003
Iter:2360 Loss:0.743 Accuracy:0.90,0.70,0.60,0.80  Learning_rate:0.0003
Iter:2380 Loss:0.622 Accuracy:0.90,0.65,0.65,0.85  Learning_rate:0.0003
Iter:2400 Loss:0.578 Accuracy:0.95,0.75,0.65,0.85  Learning_rate:0.0003
Iter:2420 Loss:0.769 Accuracy:0.95,0.80,0.50,0.65  Learning_rate:0.0003
Iter:2440 Loss:0.779 Accuracy:0.80,0.70,0.55,0.75  Learning_rate:0.0003
Iter:2460 Loss:0.695 Accuracy:0.90,0.60,0.65,0.80  Learning_rate:0.0003
Iter:2480 Loss:0.517 Accuracy:0.85,0.75,0.75,1.00  Learning_rate:0.0003
Iter:2500 Loss:0.853 Accuracy:0.80,0.75,0.55,0.85  Learning_rate:0.0003
Iter:2520 Loss:0.631 Accuracy:0.85,0.75,0.65,0.75  Learning_rate:0.0003
Iter:2540 Loss:0.615 Accuracy:1.00,0.60,0.55,0.85  Learning_rate:0.0003
Iter:2560 Loss:0.869 Accuracy:0.90,0.40,0.55,0.75  Learning_rate

Iter:4580 Loss:0.115 Accuracy:1.00,0.95,0.90,1.00  Learning_rate:0.0001
Iter:4600 Loss:0.292 Accuracy:0.95,0.80,0.80,0.95  Learning_rate:0.0001
Iter:4620 Loss:0.183 Accuracy:1.00,0.95,0.90,0.95  Learning_rate:0.0001
Iter:4640 Loss:0.133 Accuracy:0.95,0.95,0.95,1.00  Learning_rate:0.0001
Iter:4660 Loss:0.170 Accuracy:1.00,0.85,0.90,0.85  Learning_rate:0.0001
Iter:4680 Loss:0.109 Accuracy:1.00,0.95,1.00,0.90  Learning_rate:0.0001
Iter:4700 Loss:0.078 Accuracy:1.00,0.95,0.95,1.00  Learning_rate:0.0001
Iter:4720 Loss:0.156 Accuracy:0.95,0.90,1.00,0.95  Learning_rate:0.0001
Iter:4740 Loss:0.202 Accuracy:1.00,0.85,0.85,1.00  Learning_rate:0.0001
Iter:4760 Loss:0.144 Accuracy:1.00,0.95,0.95,0.90  Learning_rate:0.0001
Iter:4780 Loss:0.155 Accuracy:1.00,1.00,0.90,0.90  Learning_rate:0.0001
Iter:4800 Loss:0.216 Accuracy:1.00,0.85,0.90,0.85  Learning_rate:0.0001
Iter:4820 Loss:0.177 Accuracy:0.95,0.90,1.00,1.00  Learning_rate:0.0001
Iter:4840 Loss:0.189 Accuracy:1.00,1.00,0.85,0.95  Learning_rate

Iter:6860 Loss:0.126 Accuracy:0.95,0.90,1.00,1.00  Learning_rate:0.0000
Iter:6880 Loss:0.155 Accuracy:1.00,0.85,1.00,0.95  Learning_rate:0.0000
Iter:6900 Loss:0.136 Accuracy:1.00,0.90,0.95,1.00  Learning_rate:0.0000
Iter:6920 Loss:0.102 Accuracy:1.00,1.00,0.85,1.00  Learning_rate:0.0000
Iter:6940 Loss:0.179 Accuracy:0.90,0.90,0.95,1.00  Learning_rate:0.0000
Iter:6960 Loss:0.185 Accuracy:1.00,0.90,0.95,0.90  Learning_rate:0.0000
Iter:6980 Loss:0.107 Accuracy:1.00,1.00,0.90,1.00  Learning_rate:0.0000
Iter:7000 Loss:0.216 Accuracy:1.00,0.95,0.85,0.95  Learning_rate:0.0000
Iter:7020 Loss:0.138 Accuracy:0.95,0.95,1.00,0.95  Learning_rate:0.0000
Iter:7040 Loss:0.135 Accuracy:0.95,0.90,1.00,1.00  Learning_rate:0.0000
Iter:7060 Loss:0.143 Accuracy:1.00,0.95,0.90,0.90  Learning_rate:0.0000
Iter:7080 Loss:0.076 Accuracy:0.95,1.00,0.95,1.00  Learning_rate:0.0000
Iter:7100 Loss:0.088 Accuracy:1.00,0.95,0.95,1.00  Learning_rate:0.0000
Iter:7120 Loss:0.066 Accuracy:1.00,0.95,0.95,1.00  Learning_rate

Iter:9140 Loss:0.045 Accuracy:0.95,1.00,0.95,1.00  Learning_rate:0.0000
Iter:9160 Loss:0.048 Accuracy:1.00,1.00,1.00,1.00  Learning_rate:0.0000
Iter:9180 Loss:0.031 Accuracy:1.00,1.00,0.95,1.00  Learning_rate:0.0000
Iter:9200 Loss:0.041 Accuracy:1.00,1.00,0.95,1.00  Learning_rate:0.0000
Iter:9220 Loss:0.074 Accuracy:1.00,0.95,1.00,0.95  Learning_rate:0.0000
Iter:9240 Loss:0.108 Accuracy:1.00,1.00,0.95,0.90  Learning_rate:0.0000
Iter:9260 Loss:0.037 Accuracy:1.00,1.00,0.90,1.00  Learning_rate:0.0000
Iter:9280 Loss:0.074 Accuracy:1.00,1.00,0.90,1.00  Learning_rate:0.0000
Iter:9300 Loss:0.093 Accuracy:1.00,0.95,1.00,0.95  Learning_rate:0.0000
Iter:9320 Loss:0.079 Accuracy:1.00,0.95,1.00,0.95  Learning_rate:0.0000
Iter:9340 Loss:0.020 Accuracy:1.00,1.00,1.00,1.00  Learning_rate:0.0000
Iter:9360 Loss:0.076 Accuracy:1.00,1.00,1.00,0.90  Learning_rate:0.0000
Iter:9380 Loss:0.145 Accuracy:1.00,0.95,0.95,0.90  Learning_rate:0.0000
Iter:9400 Loss:0.084 Accuracy:0.95,0.90,0.95,1.00  Learning_rate